In [1]:
import torch

import pandas as pd
import numpy as np

from tqdm import tqdm
from model import CryptoTransformer, MyDataset
from utils import weighted_corr

In [2]:
processed_test = pd.read_csv('./processed_data/processed_test.gz')
asset_list = pd.read_csv('./g-research-crypto-forecasting/asset_details.csv')
asset_list.set_index('Asset_ID', inplace=True)
processed_test['Weight'] = processed_test['Asset_ID'].apply(lambda x: asset_list.loc[x]['Weight'])

In [45]:
net = torch.load('./checkpoint/size_128_128_nheads_4_nlayers_2/epoch_2_step_8.pt')
test_dataset = MyDataset(processed_test)

In [46]:
predictions = []
net.eval()
net = net.cpu()
with torch.no_grad():
    with tqdm(test_dataset) as t:
        for step, (x, y, num_assets) in enumerate(t):
            mask = torch.arange(14)
            mask = mask < num_assets
            y_pred = net(x, mask)
            y_pred = y_pred[:num_assets]
            predictions.append(y_pred)

100%|██████████| 81010/81010 [02:05<00:00, 646.30it/s]


In [47]:
predictions = torch.cat(predictions).numpy()
processed_test['Predictions'] = predictions

In [48]:
p_corr = np.corrcoef(processed_test['Predictions'], processed_test['Target'])[1][0]
w_corr = weighted_corr(processed_test['Predictions'], processed_test['Target'], processed_test['Weight'])
print(f'Pearson Correlation: {p_corr:.2e}, Weighted Pearson Correlation: {w_corr:.2e}')

Pearson Correlation: 4.30e-03, Weighted Pearson Correlation: 4.37e-03
